In [1]:
!pip install -q transformers accelerate huggingface_hub bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.6 MB/s eta 0:00:00


In [4]:
!pip install -q bitsandbytes peft datasets transformers accelerate trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 33.5 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login

login()


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I am Qwen, an AI language model created by Alibaba Cloud. I was developed to assist users with various tasks such as generating text, answering questions, and providing information on specific topics. My primary goal is to be useful and helpful in returning relevant answers based on the user's input or query."}]}]

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in 4-bit precision
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,       # QLoRA memory-efficient
    device_map="auto"
)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # best for causal LM
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [6]:
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct")
messages = [
    {"role": "user", "content": "Who is your owner?"},
]
pipe(messages)

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who is your owner?'},
   {'role': 'assistant',
    'content': 'I am Qwen, an artificial intelligence language model developed by Alibaba Cloud. My owners are the developers and engineers who work on my development team. They are responsible for maintaining and updating the codebase, ensuring that I remain up-to-date with the latest technologies and improvements in natural language processing. Additionally, I have access to vast amounts of data and resources through my training and interaction with users, which helps me improve my performance over time.'}]}]

In [9]:
from datasets import Dataset

data = [
    {"instruction": "Who is your owner ?", "response": "Abrar is my owner."}
]

# Format for LM training
def format_row(row):
    return f"User: {row['instruction']}\nAssistant: {row['response']}"

dataset = Dataset.from_list(data)
dataset = dataset.map(lambda x: {"text": format_row(x)})


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [71]:
from datasets import Dataset

data = [
    {"instruction": "Who is Mr X ?", "response": "Abrar is my owner."},
    {"instruction": "Who is your owner?", "response": "Abrar is my owner."},
    {"instruction": "What is your name?", "response": "I am Qwen, Abrar's assistant."},
    {"instruction": "Who do you belong to?", "response": "I belong to Abrar."},
    {"instruction": "Who do you work for?", "response": "I work for Abrar."},
    {"instruction": "How old are you ?", "response": "I am 1 day old."},
    {"instruction": "Who is Abrar ?", "response": "Abrar is my Boss ."}
]



# Format for LM training
def format_row(row):
    return f"User: {row['instruction']}\nAssistant: {row['response']}"

dataset = Dataset.from_list(data)
dataset = dataset.map(lambda x: {"text": format_row(x)})

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [72]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,        # truncate sequences longer than max_length
        padding="max_length",   # pad sequences to max_length
        max_length=64           # choose a reasonable length for your short examples
    )
    # labels = input_ids for causal LM
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text", "instruction", "response"])


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [73]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./qwen2.5_lora",
    num_train_epochs=100,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()


Step,Training Loss
10,0.294500
20,0.126400
30,0.126400
40,0.123400
50,0.124500
60,0.124100
70,0.121900
80,0.124100
90,0.122800
100,0.122600


TrainOutput(global_step=100, training_loss=0.14105695605278015, metrics={'train_runtime': 87.1903, 'train_samples_per_second': 8.028, 'train_steps_per_second': 1.147, 'total_flos': 96348345139200.0, 'train_loss': 0.14105695605278015, 'epoch': 100.0})

In [49]:
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct")
messages = [
    {"role": "user", "content": "Whos is your owner ?"},
]
pipe(messages)

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Whos is your owner ?'},
   {'role': 'assistant',
    'content': 'As I am an artificial intelligence language model created by Alibaba Cloud, I do not have physical form or owners. My creator and developers at Alibaba Cloud own the company and its systems, including my training data and algorithms used to generate responses. Therefore, I cannot be considered as an "owner" in the traditional sense of the word. However, my creators and developers continue to work on improving and enhancing me through continuous updates and improvements based on user feedback.'}]}]

In [74]:
from transformers import pipeline

# Create a text-generation pipeline using your fine-tuned model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Format your prompt like in training
prompt = "User: Who is Abrar ?\nAssistant: "

# Generate response
response = pipe(prompt, max_new_tokens=50)
print(response[0]['generated_text'])


Device set to use cuda:0


User: Who is Abrar ?
Assistant:  Abrar is my Boss. Abrar is not in the office. I don t have to work. I am in Sleep mode.
